# (2m) Detected true units

author: steeve.laquitaine@epfl.ch


**method**

* first 10 minutes
* single-units whenever possible

## Setup

activate `spikeinterf..`

In [35]:
%load_ext autoreload
%autoreload 2
import os 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import spikeinterface as si
from scipy import stats

# set project path
proj_path = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/"
os.chdir(proj_path)

from src.nodes import utils
from src.nodes.utils import get_config
from src.nodes.metrics import quality
from src.nodes.analysis.accuracy import accuracy as acc
from src.nodes.metrics.quality import get_scores

# PARAMETERS
DUR = 600 # 10 minutes recording
DT = 1.3 # ms (optimized)
THR_GOOD = 0.8

# DATASETS

# NPX PROBE
# Synthetic (10m)
cfg_nb, _ = get_config("buccino_2020", "2020").values()
GT_nb_10m = cfg_nb["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
KS4_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
KS3_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
KS2_5_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
KS2_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
KS_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort"]["10m"]["output"]
HS_nb_10m = cfg_nb["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
REC_nb = cfg_nb["probe_wiring"]["full"]["output"]

# biophy spont (10m)
cfg_ns, _ = get_config("silico_neuropixels", "concatenated").values()
KS4_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
KS3_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
KS2_5_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
KS2_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
KS_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort"]["10m"]["output"]
HS_ns_10m = cfg_ns["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
GT_ns_10m = cfg_ns["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
REC_ns = cfg_ns["probe_wiring"]["full"]["output"]

# biophy evoked
cfg_e, _ = get_config("silico_neuropixels", "stimulus").values()
KS4_e_10m = cfg_e["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
KS3_e_10m = cfg_e["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
KS2_5_e_10m = cfg_e["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
KS2_e_10m = cfg_e["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
KS_e_10m = cfg_e["sorting"]["sorters"]["kilosort"]["10m"]["output"]
HS_e_10m = cfg_e["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
GT_e_10m = cfg_e["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
REC_e = cfg_e["probe_wiring"]["full"]["output"]

# DENSE PROBE 
# depth 1
cfg_ds1, _ = get_config("silico_horvath", "concatenated/probe_1").values()
K4_d1 = cfg_ds1["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
K3_d1 = cfg_ds1["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
K25_d1 = cfg_ds1["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
K2_d1 = cfg_ds1["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
K_d1 = cfg_ds1["sorting"]["sorters"]["kilosort"]["10m"]["output"]
H_d1 = cfg_ds1["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
R_d1 = cfg_ds1["probe_wiring"]["full"]["output"]
T_d1 = cfg_ds1["sorting"]["simulation"]["ground_truth"]["10m"]["output"]

# depth 2
cfg_ds2, _ = get_config("silico_horvath", "concatenated/probe_2").values()
K4_d2 = cfg_ds2["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
K3_d2 = cfg_ds2["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
K25_d2 = cfg_ds2["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
K2_d2 = cfg_ds2["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
K_d2 = cfg_ds2["sorting"]["sorters"]["kilosort"]["10m"]["output"]
H_d2 = cfg_ds2["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
R_d2 = cfg_ds2["probe_wiring"]["full"]["output"]
T_d2 = cfg_ds2["sorting"]["simulation"]["ground_truth"]["10m"]["output"]

# depth 3
cfg_ds3, _ = get_config("silico_horvath", "concatenated/probe_3").values()
K4_d3 = cfg_ds3["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
K3_d3 = cfg_ds3["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
K25_d3 = cfg_ds3["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
K2_d3 = cfg_ds3["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
K_d3 = cfg_ds3["sorting"]["sorters"]["kilosort"]["10m"]["output"]
H_d3 = cfg_ds3["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
R_d3 = cfg_ds3["probe_wiring"]["full"]["output"]
T_d3 = cfg_ds3["sorting"]["simulation"]["ground_truth"]["10m"]["output"]

# SAVE PATH
save_path = "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/analysis/indegree/"

# axes
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 6  # 5-7 with Nature neuroscience as reference
plt.rcParams["lines.linewidth"] = 0.5 # typically between 0.5 and 1
plt.rcParams["axes.linewidth"] = 0.5 #1
plt.rcParams["axes.spines.top"] = False
plt.rcParams["xtick.major.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["xtick.minor.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["ytick.major.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["ytick.minor.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["xtick.major.size"] = 3.5 * 1.1
plt.rcParams["xtick.minor.size"] = 2 * 1.1
plt.rcParams["ytick.major.size"] = 3.5 * 1.1
plt.rcParams["ytick.minor.size"] = 2 * 1.1
# legend
legend_cfg = {"frameon": False, "handletextpad": 0.5}
tight_layout_cfg = {"pad": 0.001}
LG_FRAMEON = False              # no legend frame


# save paths


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-09-25 16:19:18,244 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-25 16:19:18,276 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-09-25 16:19:18,278 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-25 16:19:18,313 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-09-25 16:19:18,314 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-25 16:19:18,360 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-09-25 16:19:18,361 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-25 16:19:18,387 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-09-25 16:19:18,389 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-25 16:19:18,419 - root - utils.py - get_config - INFO - R

In [72]:
SAVE = True


def save_detected_true_units(exp, SortingTrue, k4, k3, k25, k2, ks, hs, dt):

    ## KS4
    scores4 = get_scores(SortingTrue, si.load_extractor(k4), dt)
    pd.DataFrame(scores4.max(axis=1) >= 0.8).to_csv(
        save_path + f"{exp}_k4.csv", index=False
    )

    # KS3
    scores3 = get_scores(SortingTrue, si.load_extractor(k3), dt)
    pd.DataFrame(scores3.max(axis=1) >= 0.8).to_csv(
        save_path + f"{exp}_k3.csv", index=False
    )
    # KS2.5
    scores25 = get_scores(SortingTrue, si.load_extractor(k25), dt)
    pd.DataFrame(scores25.max(axis=1) >= 0.8).to_csv(
        save_path + f"{exp}_k25.csv", index=False
    )
    # KS2
    scores2 = get_scores(SortingTrue, si.load_extractor(k2), dt)
    pd.DataFrame(scores2.max(axis=1) >= 0.8).to_csv(
        save_path + f"{exp}_k2.csv", index=False
    )
    # KS
    scores1 = get_scores(SortingTrue, si.load_extractor(ks), dt)
    pd.DataFrame(scores1.max(axis=1) >= 0.8).to_csv(
        save_path + f"{exp}_k1.csv", index=False
    )
    # HS
    scoresH = get_scores(SortingTrue, si.load_extractor(hs), dt)
    pd.DataFrame(scoresH.max(axis=1) >= 0.8).to_csv(
        save_path + f"{exp}_hs.csv", index=False
    )

In [73]:
# spontaneous
SortingTrue = si.load_extractor(GT_ns_10m)
save_detected_true_units(
    "ns",
    SortingTrue,
    KS4_ns_10m,
    KS3_ns_10m,
    KS2_5_ns_10m,
    KS2_ns_10m,
    KS2_ns_10m,
    HS_ns_10m,
    DT,
)

SortingTrue = si.load_extractor(GT_e_10m)
# evoked
save_detected_true_units(
    "e",
    SortingTrue,
    KS4_e_10m,
    KS3_e_10m,
    KS2_5_e_10m,
    KS2_e_10m,
    KS2_e_10m,
    HS_e_10m,
    DT,
)

In [80]:
# unit-testing
units = pd.read_csv(save_path + "ns_k4.csv").values.flatten()
scores4 = get_scores(si.load_extractor(GT_ns_10m), si.load_extractor(KS4_ns_10m), DT)
all(scores4.max(axis=1).iloc[np.where(units)[0]] >= 0.8)

units = pd.read_csv(save_path + "ns_k3.csv").values.flatten()
scores3 = get_scores(si.load_extractor(GT_ns_10m), si.load_extractor(KS3_ns_10m), DT)
all(scores3.max(axis=1).iloc[np.where(units)[0]] >= 0.8)

True